<a href="https://colab.research.google.com/github/Diegostvz/visualizacion-datos/blob/main/ejercicio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
pip install plotly_express

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [11]:
df = pd.read_csv("sample_data/ventas_composicion.csv")
print(df)

          Fecha Categoría  Ventas  Porcentaje
0    2020-01-01         A    8270       36.41
1    2020-01-01         B    1860        8.19
2    2020-01-01         C    6390       28.14
3    2020-01-01         D    6191       27.26
4    2020-02-01         A    6734       32.23
..          ...       ...     ...         ...
235  2024-11-01         D    1098        5.31
236  2024-12-01         A    3200       17.23
237  2024-12-01         B    3961       21.33
238  2024-12-01         C    5973       32.17
239  2024-12-01         D    5433       29.26

[240 rows x 4 columns]


## ¿Cómo ha evolucionado la cuota de mercado de cada categoría a lo largo del tiempo?

- mensaje: mostrar la evolución de la variable porcentaje de cada categoria con respecto al tiempo(la fecha en este caso)

- los datos a explicar se componen en 4 variables son: fecha(dia en que se produjo la venta), categoría(a que clase pertenece las ventas), ventas(numero de ventas del dia), porcentaje(cuota con repecto al total)

- persona: el gráfico va dirigido al profesor de la asignatura visualizacion de datos

- limitantes: tiempo para realizar el ejercicio y formato del mismo ya que hay muchas opciones


Se elige el grafico filled_area_plot ya que permite representar la evolucion temporal de una variable con respecto a un total(siendo la suma de dicha variable = 100%)

In [13]:
fig = px.area(df, x="Fecha", y="Porcentaje", color="Categoría", title="Evolución cuota de mercado con el tiempo")
fig.show()

## ¿Qué categoría tiene el comportamiento más estacional a lo largo del año?

- mensaje: mostrar el elemento más estacional, es decir el que menos variación tiene entre el maximo valor y el minimo valor en un intervalo de tiempo

- los datos a explicar son los mismos que en el primer apartado

- persona: el gráfico va dirigido al profesor de la asignatura visualizacion de datos

- limitantes: tiempo para realizar el ejercicio y formato del mismo ya que hay muchas opciones

Se crea una variable que recoja la varianza anual de cada categoria
Se elige el grafico de lineas clásico para mostrar dicha varianza en cada año. La categoria que menos varie en el tiempo, es decir que menos pendiente acumulada tenga a lo largo de tiempo será el más estacional.

In [26]:
df['Fecha'] = pd.to_datetime(df['Fecha'])
df['Año'] = df['Fecha'].dt.year
varianza_anual = df.groupby(['Año', 'Categoría'])['Porcentaje'].var().unstack()
varianza_anual

Categoría,A,B,C,D
Año,,,,
2020,44.781190,135.893133,95.803800,64.803984
2021,75.102824,119.235354,97.838136,148.394609
2022,73.670918,155.180772,129.431275,98.773354
2023,112.947063,116.031372,116.203400,122.294555
2024,149.274852,179.097954,85.409573,104.690699


In [30]:
fig = px.line(varianza_anual, title="Varianza anual de cada categoría")
fig.show()


## ¿En qué momentos una categoría supera a otra en cuota de ventas?

- mensaje: mostrar la evolucion de la cuota de ventas haciendo enfasis en los cruces de las mismas

- los datos a explicar son los mismos que en el primer apartado

- persona: el gráfico va dirigido al profesor de la asignatura visualizacion de datos

- limitantes: tiempo para realizar el ejercicio y formato del mismo ya que hay muchas opciones

Se divide por año y se hace un bump chart en cada uno

In [36]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

df['Fecha'] = pd.to_datetime(df['Fecha'])
df['Año'] = df['Fecha'].dt.year
df['Rank'] = df.groupby('Fecha')['Porcentaje'].rank(method='dense', ascending=False)

años = sorted(df['Año'].unique())
n_años = len(años)

fig = make_subplots(rows=n_años, cols=1, shared_xaxes=False, vertical_spacing=0.05,
                    subplot_titles=[f"Año {a}" for a in años])

for i, año in enumerate(años, 1):
    df_año = df[df['Año'] == año]
    for cat in df_año['Categoría'].unique():
        df_cat = df_año[df_año['Categoría'] == cat]
        fig.add_trace(go.Scatter(x=df_cat['Fecha'], y=df_cat['Rank'], mode='lines+markers', name=cat,
                                 showlegend=(i==1)), row=i, col=1)

    fecha_min = df_año['Fecha'].min()
    fecha_max = df_año['Fecha'].max()
    fig.update_xaxes(range=[fecha_min, fecha_max], row=i, col=1)

# se invierte eje Y para que rank=1 esté arriba
fig.update_yaxes(autorange='reversed', dtick=1)

fig.update_layout(height=300*n_años, title_text="Bump chart por año con escala X individual",
                  hovermode="x unified")

fig.show()


## ¿Cómo cambia la participación de las categorías si agrupamos los datos por año en lugar de por mes?